In [1]:
%reload_ext autoreload
%autoreload 2

# SO-100 Robot Control Demo

This notebook demonstrates how to drive the SO-100 arm using the `SO100RobotClient`.
Only control endpoints are covered; AI control endpoints are intentionally omitted.

## Requirements

- A running instance of the robot control server (defaults to `http://localhost:8000`).
- Network connectivity between this notebook and the server.
- Physical access to the robot if you plan to execute motion commands. Ensure it is safe to move before running the cells below.

In [43]:
from so100_robot import SO100RobotClient

BASE_URL = "http://localhost:80"  # Update if your server runs elsewhere
ROBOT_ID = 0  # Replace with the target robot id

robot = SO100RobotClient(BASE_URL, robot_id=ROBOT_ID)
robot

## Verify Server Status

Confirm that the API is reachable before issuing motion commands.

In [423]:
robot.request("GET", "/status")

{'status': 'ok',
 'name': 'Alireza',
 'robots': ['so-100'],
 'robot_status': [{'name': 'so-100',
   'robot_type': 'manipulator',
   'device_name': '5A46085878',
   'temperature': [{'current': 25.0, 'max': 70.0},
    {'current': 33.0, 'max': 70.0},
    {'current': 30.0, 'max': 70.0},
    {'current': 25.0, 'max': 70.0},
    {'current': 25.0, 'max': 70.0},
    {'current': 31.0, 'max': 70.0}]}],
 'cameras': {'cameras_status': [{'camera_id': 0,
    'is_active': True,
    'camera_type': 'classic',
    'width': 640,
    'height': 360,
    'fps': 30}],
  'is_stereo_camera_available': False,
  'realsense_available': False,
  'video_cameras_ids': [0]},
 'version_id': '0.3.134',
 'is_recording': False,
 'ai_running_status': 'stopped',
 'leader_follower_status': False,
 'server_ip': '10.156.69.24',
 'server_port': 80}

## Initialize the Robot

Moves the arm to the default starting pose, which is required before other move commands.

In [463]:
robot.initialize()

{'status': 'ok', 'message': None}

## Absolute Move

Command the end-effector to go to a position and orientation expressed in centimeters and degrees.

In [184]:
state = robot.read_end_effector(0)
state

{'x': 0.6539881229400635,
 'y': 0.10558032989501953,
 'z': -0.1002882868051529,
 'rx': 271.62395962687475,
 'ry': -31.75834837226449,
 'rz': 105.80806200792257,
 'open': 0.5}

In [50]:
robot.move_absolute(
    x= 8.01367461681366,
    y= -9.313225746154785e-10,
    z= -0.05518127977848053,
    rx= 0.1694453776592496,
    ry= 0.000008277720443196744,
    rz= -0.00026973307779949333,
    open= 0,
    angle_unit="rad",  # set to "rad" when supplying radians
)


{'status': 'ok', 'message': None}

In [187]:
robot.move_absolute(
  table=[{
    'x': 0.6539881229400635,
 'y': 0.10558032989501953,
 'z': -0.1002882868051529,
 'rx': 271.62395962687475,
 'ry': -31.75834837226449,
 'rz': 105.80806200792257,}],
  angle_unit="degrees",  # set to "rad" when supplying radians
)

[{'status': 'ok', 'message': None}]

## Batch Absolute Moves

Pass a list of dictionaries to call `move_absolute` sequentially. Values provided
above the table act as defaults for any missing keys.

In [53]:
robot.move_absolute(
    table=[
        {"x": 25, "z": 15, "open": 1},
        {"x": 20, "z": 20, "open": 0},
    ],
    angle_unit="degrees",
)


[{'status': 'ok', 'message': None}, {'status': 'ok', 'message': None}]

## Relative Move

Apply deltas to the current end-effector pose. Useful for jogging the arm.

In [54]:
robot.move_relative(z=5, open=0)

{'status': 'ok', 'message': None}

## Teleoperation Frame

Send a single MetaQuest-style frame. Repeated calls can be used to stream joystick-style input.

In [56]:
robot.teleop_control(
    x=25,
    y=5,
    z=12,
    rx=10,
    ry=0,
    rz=0,
    open=0.5,
    direction_x=0.0,
    direction_y=0.0,
    angle_unit="degrees",  # switch to "rad" if providing radians
)


{'status': 'ok', 'message': None}

## Inspect Joint State

Retrieve the current joint angles reported by the controller.

In [195]:
joints = robot.read_joints(unit="rad")  # switch to "rad" or "motor_units" as needed
joints

{'angles': [0.3390925403874697,
  0.7395592962296851,
  -0.6474979730475666,
  1.2857898137769215,
  1.4821873032321076,
  0.17338215865965648],
 'unit': 'rad'}

## Write Joint Targets

Set joint angles directly. Adjust the angle list to match your robot's configuration.

In [ ]:
import math

LIMIT = math.pi/2  # joint limits [-pi/2, pi/2]

def ik(x, y, z, a1, a2, *, last_drop=0.0, shoulder_z=0.0,
         degrees=False, tol=1e-9):
    """
    Frame: +y forward, +x left, +z up. Base at z=0.
    Inputs:
      x,y = target TCP XY
      z   = target TCP Z (use your table height here)
      a1  = shoulder->elbow length
      a2  = elbow->wrist length
      last_drop = fixed length of locked last link pointing toward -z when the first two links are along +y
      shoulder_z = shoulder axis height above base (0 if base at z=0)
    Joint mapping (your convention):
      θ1 = atan2(x,y)
      θ2 = π/2 - θs         (θs = shoulder from +y)
      θ3 = ψ - π/2          (ψ = elbow internal angle, 0 when straight)
    All joints constrained to [-π/2, π/2].
    """
    def saturate(a): return max(-LIMIT, min(LIMIT, a))

    # Yaw
    t1_raw = math.atan2(x, y)
    t1 = saturate(t1_raw)

    # Project target into the chosen yaw plane
    cy, sy = math.cos(t1), math.sin(t1)
    r = x*sy + y*cy                # horizontal distance along yaw
    zp = z - shoulder_z            # height from shoulder

    def solve_planar(rr, zz, elbow_sign):
        d = math.hypot(rr, zz)
        c = (d*d - a1*a1 - a2*a2) / (2*a1*a2)
        if c < -1.0000001 or c > 1.0000001:
            return None
        c = max(-1.0, min(1.0, c))
        s = math.sqrt(max(0.0, 1.0 - c*c))
        if elbow_sign < 0: s = -s
        ψ = math.atan2(s, c)
        φ = math.atan2(zz, rr)
        β = math.atan2(a2*s, a1 + a2*c)
        θs = φ - β
        return θs, ψ

    def solve_with_drop(elbow_sign):
        # iterate once to account for last_drop direction depending on θs
        res = solve_planar(r, zp, elbow_sign)
        if res is None: return None
        θs, ψ = res
        for _ in range(2):
            # locked last link direction in plane: (-sin θs, -cos θs)
            vr, vz = -math.sin(θs), -math.cos(θs)
            rw = r - last_drop*vr
            zw = zp - last_drop*vz
            res2 = solve_planar(rw, zw, elbow_sign)
            if res2 is None: return None
            θs2, ψ2 = res2
            if abs(θs2-θs) < tol and abs(ψ2-ψ) < tol:
                θs, ψ = θs2, ψ2
                break
            θs, ψ = θs2, ψ2
        # Map to your joints
        t2 = math.pi/2 - θs
        t3 = ψ - math.pi/2
        return t2, t3

    # try both elbow branches
    candidates = []
    for sign in (+1, -1):
        res = solve_with_drop(sign)
        if res:
            t2, t3 = res
            candidates.append((t1, t2, t3))

    # pick in-limits first, else closest after clamping
    for sol in candidates:
        if all(-LIMIT <= a <= LIMIT for a in sol):
            return tuple(a*180/math.pi for a in sol) if degrees else sol

    if candidates:
        best = min(candidates,
                   key=lambda s: sum(abs(saturate(a)-a) for a in s))
        best = tuple(saturate(a) for a in best)
        return tuple(a*180/math.pi for a in best) if degrees else best

    raise ValueError("Unreachable with given geometry and table height.")

In [513]:
a1 = 0.15
a2 = 0.15
last_drop = -0.18

In [ ]:
print(ik(0.05, 0.05, 0, a1, a2, last_drop=last_drop, degrees=True))

(45.0, 56.77652338136277, -90.0)


In [515]:
robot.initialize()

{'status': 'ok', 'message': None}

In [ ]:
# Yes
robot.write_joints(
    angles=  [*ik(0, 0.2, 0, a1, a2, last_drop=last_drop),
  -math.pi/2,#1.2857898137769215,
  0, #math.pi/2,#1.4821873032321076,
  0.],
    unit="rad",
)

{'status': 'ok', 'message': None}

In [58]:
robot.write_joints(
    angles=[0.0 for _ in joints.get("angles", [])],
    unit="rad",
)

{'status': 'ok', 'message': None}

In [131]:
robot.write_joints(
    angles=[0.25,
  0.0889926124093812,
  0.26,
  0.7840556024343758,
  1.5707963268,
  0.],
    unit="rad",
)

{'status': 'ok', 'message': None}

## Cleanup

Return the arm to a safe pose or disable torque when finishing a session.

In [ ]:
robot.sleep()
# robot.toggle_torque(False)  # Uncomment to disable torque explicitly